In [ ]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
import wandb
import datetime
import pytz

In [17]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('../../../baseline/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../../../dataset/'
dataset_idx = 4

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = f'../dataset/train{dataset_idx}.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = f'../dataset/valid{dataset_idx}.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

# wandb setting
kst = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(kst)
date = now.strftime('%Y_%m_%d_%H_%M_%S_')
project_name = "Mini"
exp_name = date + f"FasterRCNN_Trim5000_SGKF_{dataset_idx}"
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project': project_name, 'name' : exp_name},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=0,
         bbox_score_thr=0.3)]

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/' + exp_name

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [18]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [19]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3913, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3151  | 1 [Paper]     | 5049  | 2 [Paper pack]  | 717   | 3 [Metal]   | 742   | 4 [Glass]    | 783   |
| 5 [Plastic]       | 2335  | 6 [Styrofoam] | 1004  | 7 [Plastic bag] | 4114  | 8 [Battery] | 127   | 9 [Clothing] | 373   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [20]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2023-05-12 08:03:03,324 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2023-05-12 08:03:03,325 - mmcv - INFO - load model from: torchvision://resnet50
2023-05-12 08:03:03,326 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2023-05-12 08:03:03,576 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2023-05-12 08:03:03,607 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2023-05-12 08:03:03,635 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2023-05-12 08:03:03,650 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [21]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2023-05-12 08:03:04,081 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-12 08:03:04,108 - mmdet - INFO - Start running, host: root@900ed28a7422, work_dir: /opt/ml/level2_objectdetection-cv-06/Hyunjin Jung/code/work_dirs/2023_05_12_17_03_01_FasterRCNN_Trim5000_SGKF_4
2023-05-12 08:03:04,109 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook               

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2023-05-12 08:03:08,103 - mmdet - WARNING - No meta information found in the runner. 
2023-05-12 08:03:21,946 - mmdet - INFO - Epoch [1][50/979]	lr: 1.978e-03, eta: 0:53:56, time: 0.277, data_time: 0.053, memory: 2106, loss_rpn_cls: 0.4283, loss_rpn_bbox: 0.0437, loss_cls: 0.5641, acc: 89.8037, loss_bbox: 0.1044, loss: 1.1405, grad_norm: 6.6862
2023-05-12 08:03:33,462 - mmdet - INFO - Epoch [1][100/979]	lr: 3.976e-03, eta: 0:49:13, time: 0.230, data_time: 0.008, memory: 2106, loss_rpn_cls: 0.1676, loss_rpn_bbox: 0.0488, loss_cls: 0.3166, acc: 94.1221, loss_bbox: 0.2184, loss: 0.7514, grad_norm: 2.8966
2023-05-12 08:03:45,499 - mmdet - INFO - Epoch [1][150/979]	lr: 5.974e-03, eta: 0:48:10, time: 0.241, data_time: 0.008, memory: 2106, loss_rpn_cls: 0.1333, loss_rpn_bbox: 0.0479, loss_cls: 0.3232, acc: 93.8389, loss_bbox: 0.2367, loss: 0.7411, grad_norm: 2.5763
2023-05-12 08:03:57,506 - mmdet - INFO - Epoch [1][200/979]	lr: 7.972e-03, eta: 0:47:32, time: 0.240, data_time: 0.007, memory: 2

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 970/970, 23.2 task/s, elapsed: 42s, ETA:     0s

2023-05-12 08:07:45,516 - mmdet - INFO - Evaluating bbox...


...

2023-05-12 09:01:39,026 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.96s).
Accumulating evaluation results...


2023-05-12 09:01:44,501 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.354
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.208
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.242
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.410
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.410
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.410
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.038
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.177
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.09s).
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.91s).
Accumulating evaluation results...
DONE (t=1.09s).


learning_rate,▃▅█████████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▆▃▁▃▃▃▄▂▄▃▁▄▃▄▃▃▁▄▁▃▅▅▅▅▂▄▄▅█▆▅█▇▆█▆▅▆█▇
train/grad_norm,██▆▄▃▅▃▄▂▃▃▃▃▃▂▆▆▃▆▃▂▄▄▂▅▄▃▂▁▄▃▂▂▄▂▄▄▄▂▄
train/loss,▆██▆▆▆▅▇▅▅▇▅▅▅▅▆▆▄▆▅▄▄▄▄▆▄▃▃▁▃▄▁▂▃▂▃▃▂▁▂
train/loss_bbox,▆██▆▇▆▄▇▄▅█▅▆▅▆▆▇▄▇▅▄▄▄▄▇▅▄▄▁▃▅▁▃▅▂▄▄▂▂▃
train/loss_cls,▆██▇▇▇▅▇▆▅▇▅▅▅▅▆▇▄▆▅▄▄▄▄▆▄▄▃▁▃▃▁▁▃▁▂▃▂▁▂
train/loss_rpn_bbox,▄▆█▃▄▅▄▅▄▄▅▄▅▃▄▄▄▃▅▄▃▃▃▃▆▄▂▃▁▃▃▁▂▃▂▂▃▁▂▃
train/loss_rpn_cls,▆█▆▄▄▅▅▅▃▃▃▄▄▃▃▄▃▂▃▃▂▂▂▃▃▃▁▂▁▂▂▁▁▂▁▁▂▁▁▁
val/bbox_mAP,▁▂▃▄▅▄▅▅▇███
val/bbox_mAP_50,▁▃▃▄▅▅▅▅▇███


In [ ]:
wandb.finish()